**Fin 585R**  
**Diether**  
**Problem Set**  
**Analyst Dispersion Portfolios**  

**Purpose/Goal**

The primary purpose of this problem set is to give you a portfolio formation task that makes you go through the first four steps of our portfolio formation framework.

1. Data Preparation.<br><br>

2. Create portfolio formation or criterion variable.<br><br>

3. Bin the data based on the formation variable.<br><br>

4. Portfolio creeation using the bins.<br><br>

5. Test the historical performance and/or test a model.<br><br>

A secondary goal is to introduce another interesting portfolio strategy. It's produces a large spread in average return, and we will use it later in testing models like the CAPM.

To accomplish the programming takes, you should be able to adapt a lot of code we've used before, and apply it this situation. 

**Overview**

In this problem set you reproduce another seminal empirical result in academic finance. Specifically, you reproduce the **dispersion effect** (or the analyst disgreement effect) of Diether, Malloy, and Scherbina (2002). This empirical result spawned a large literature in academic finance, and certainly some quant funds have tried to trade on this effect.

Dispersion (or analyst disagreement) portfolios are formed based on the standard deviation of analyst eps (earnings per share) forecasts over a given period. Here the standard deviation of analyst eps forecasts is the standard deviation across analysts for a given stock and month (most stocks have between 3 to 13 analysts covering them). Diether, Malloy, and Scherbina don't use raw standard deviation. Instead, they scale the standard deviation of analyst forecasts by the absolute value of the mean forecast. Therefore for a given month ($t$), dispersion for stock $i$ is defined as the following:

$$
disp_{it} = \frac{stdev_{it}}{|mean_{it}|}
$$

DMS form dispersion portfolios using $disp_{i,t-1}$; in other words, they lag dispersion one month. In this homework you will do the same. Additionally, you will form dispersion portfolio based on lagging dispersion 3 months.

There are two datasets for this problem set. The first is the CRSP data (security prices and returns) during the period from January of 1982 to December of 2000. The second is the analyst earnings per share data from IBES. It also covers the period of January of 1982 to December of 2000. The frequency for both datasets is monthly. The stock level identifier in the IBES data is called a CUSIP. Consequently, I also included CUSIPs in the CRSP data. The CUSIP and the calender month uniquely identify the analyst earnings per share observations.

You can download the CRSP data directly using the following link: [the CRSP data](http://diether.org/prephd/08-mstk_82-00.csv). There is also a link on *Learning Suite*. The data contain the following variables:

|Variable | Description                                              |
|---------|----------------------------------------------------------|
|permno   | stock identifier                                         |
|cusip    | stock identifier also in IBES data                       |
|caldt    | calendar date (the day is not truncated to 1)            |
|ret      | monthly return                                           |
|prc      | stock price (not lagged, contemporaneous with returns)   |   


You can download the IBES data directly using the following link: [the IBES data](http://diether.org/prephd/08-ibes_eps_analyst.csv). There is also a link on *Learning Suite*. The data contain the following variables:

|Variable | Description                                          |
|---------|------------------------------------------------------|
|cusip    | stock identifier also in IBES data                   |
|caldt    | calendar date (the day is not truncated to 1)        |
|meanest  | average analyst forecast for that month/stock        |
|stdev    | standard deviation of forecasts for that month/stock |


**Tasks**

1. Form quintile based equal-weight dispersion portfolios where dispersion is lagged one month. Report summary statistics (including a t-test of whether the average return is statistically different from zero for each portfolio). Note, you should exclude low price stocks from your portfolios (price below $5). <br><br>

2. Add a spread portfolio to your dataframe of dispersion portfolios. Report summary statistics (including a t-test of whether the average return is statistically different from zero for each portfolio).<br><br>

3. Compute the average number of stocks that are in each portfolio.<br><br>

4. Form quintile based equal-weight dispersion portfolios where dispersion is lagged three month instead of one.  Report summary statistics (including a t-test of whether the average return is statistically different from zero for each portfolio). Note, you should exclude low price stocks from your portfolios (price below $5).<br><br>

5. Compare the results from (1) and (4). What do either the differences or similarities in the average return pattern tell you about the nature of this dispersion effect?

In [56]:
import pandas as pd
import numpy as np
from finance_byu.summarize import summary

In [57]:
stk = pd.read_csv('08-mstk_82-00.csv',parse_dates=['caldt'])
stk.head(5)

,permno,caldt,cusip,ret,prc,me
0,10000,1986-01-31,68391610,NaN,4.37500,16.1000
1,10000,1986-02-28,68391610,-0.257143,3.25000,11.9600
2,10000,1986-03-31,68391610,0.365385,4.43750,16.3300
3,10000,1986-04-30,68391610,-0.098592,4.00000,15.1720
4,10000,1986-05-30,68391610,-0.222656,3.10938,11.7939


In [58]:
ibes = pd.read_csv("08-ibes_eps_analyst.csv",parse_dates=['caldt'])
ibes.head(5)

,cusip,caldt,meanest,stdev
0,00000117,1982-01-14,15.36,0.78
1,00000117,1982-02-18,15.18,0.78
2,00000117,1982-03-18,15.07,0.66
3,00000117,1982-04-15,15.06,0.70
4,00000117,1982-05-20,14.78,0.71


**Hint About Merging the two Datasets**

In the datasets I've include the full calender dates of the observations. Even though the frequency for both is monthly, the timing is not the same. The CRSP data is from the last trading day in the month, and the IBES data tends to be around the middle of the month. Therefore, to merge these dataframes you need to ctreate a new date variable that only preserve uniqueness at the year-month level. Here is a shortcut way to accomplish that:

In [59]:
stk['mdt'] = stk['caldt'].values.astype('datetime64[M]')
stk.head(5)

,permno,caldt,cusip,ret,prc,me,mdt
0,10000,1986-01-31,68391610,NaN,4.37500,16.1000,1986-01-01
1,10000,1986-02-28,68391610,-0.257143,3.25000,11.9600,1986-02-01
2,10000,1986-03-31,68391610,0.365385,4.43750,16.3300,1986-03-01
3,10000,1986-04-30,68391610,-0.098592,4.00000,15.1720,1986-04-01
4,10000,1986-05-30,68391610,-0.222656,3.10938,11.7939,1986-05-01


In [60]:
ibes['mdt'] = ibes['caldt'].values.astype('datetime64[M]')
ibes.head(5)

,cusip,caldt,meanest,stdev,mdt
0,00000117,1982-01-14,15.36,0.78,1982-01-01
1,00000117,1982-02-18,15.18,0.78,1982-02-01
2,00000117,1982-03-18,15.07,0.66,1982-03-01
3,00000117,1982-04-15,15.06,0.70,1982-04-01
4,00000117,1982-05-20,14.78,0.71,1982-05-01


What is the code above doing? Pandas stores all dates with precision to the nanosecond. But numpy (the library pandas uses for its date functionality) actually includes date types for varying levels of precision (including monthly). So the above code changes the original nanosecond datetype to a monthly datetype; this causes all the information about time beyond a month to be lost and when pandas automatically reconverts the date to a nanosecond datetype the day gets set equal to one for all observations.

Now you should be able to merge the two datasets.

In [61]:
df = stk.merge(ibes, on = ['mdt','cusip'], how = 'inner')
df

,permno,caldt_x,cusip,ret,prc,me,mdt,caldt_y,meanest,stdev
0,10001,1990-04-30,39040610,0.000000,9.87500,10.14160,1990-04-01,1990-04-19,1.00,0.14
1,10001,1990-05-31,39040610,-0.012658,9.75000,10.01320,1990-05-01,1990-05-17,1.05,0.07
2,10001,1990-06-29,39040610,0.014103,9.75000,10.05230,1990-06-01,1990-06-14,1.10,0.14
3,10001,1990-07-31,39040610,0.025641,10.00000,10.31000,1990-07-01,1990-07-19,1.10,0.14
4,10001,1990-08-31,39040610,-0.050000,9.50000,9.79450,1990-08-01,1990-08-16,1.05,0.08
...,...,...,...,...,...,...,...,...,...,...
632457,93316,1989-02-28,98950D10,-0.027273,3.34375,9.01475,1989-02-01,1989-02-16,0.53,0.04
632458,93316,1989-03-31,98950D10,-0.009346,3.31250,8.93050,1989-03-01,1989-03-16,0.53,0.04
632459,93316,1989-04-28,98950D10,-0.018868,3.25000,8.76200,1989-04-01,1989-04-20,0.40,0.14
632460,93316,1989-05-31,98950D10,0.019231,3.31250,8.93050,1989-05-01,1989-05-18,0.33,0.04


In [62]:
#create dispersion var
df['disp'] = df['stdev'] / abs(df['meanest'])
#shift by 1
df['prclag'] = df.groupby('permno')['prc'].shift(1)
df['disp'] = df.groupby('permno')['disp'].shift(1)
#drop stocks with price less than 5
df = df.query("disp == disp and prclag>= 5").reset_index(drop=True)
#cut into bins
df['bins'] = df.groupby('mdt')['disp'].transform(pd.qcut,5,labels = False)
df.head(10)

,permno,caldt_x,cusip,ret,prc,me,mdt,caldt_y,meanest,stdev,disp,prclag,bins
0,10001,1990-05-31,39040610,-0.012658,9.750,10.0132,1990-05-01,1990-05-17,1.05,0.07,0.140000,9.875,3
1,10001,1990-06-29,39040610,0.014103,9.750,10.0523,1990-06-01,1990-06-14,1.10,0.14,0.066667,9.750,2
2,10001,1990-07-31,39040610,0.025641,10.000,10.3100,1990-07-01,1990-07-19,1.10,0.14,0.127273,9.750,3
3,10001,1990-08-31,39040610,-0.050000,9.500,9.7945,1990-08-01,1990-08-16,1.05,0.08,0.127273,10.000,3
4,10001,1991-04-30,39040610,0.039474,9.875,10.4083,1991-04-01,1991-04-18,1.13,0.04,0.076190,9.500,2
5,10001,1991-05-31,39040610,0.000000,9.875,10.4083,1991-05-01,1991-05-16,1.13,0.04,0.035398,9.875,1
6,10001,1991-06-28,39040610,0.078481,10.500,11.2665,1991-06-01,1991-06-20,1.13,0.03,0.035398,9.875,1
7,10001,1991-07-31,39040610,-0.035714,10.125,10.8641,1991-07-01,1991-07-18,1.13,0.03,0.026549,10.500,1
8,10001,1991-08-30,39040610,0.135802,11.500,12.3395,1991-08-01,1991-08-15,1.12,0.03,0.026549,10.125,1
9,10001,1991-09-30,39040610,0.013043,11.500,12.3395,1991-09-01,1991-09-19,1.13,0.04,0.026786,11.500,1


In [63]:
port = df.groupby(['mdt','bins'])['ret'].mean()*100
port = port.unstack(level='bins')
summary(port)

bins,0,1,2,3,4
count,2.270000e+02,227.000000,227.000000,227.000000,227.000000
mean,1.620887e+00,1.451746,1.350776,1.227299,0.850794
std,4.783370e+00,4.849491,5.202459,5.623387,6.491920
tstat,5.105421e+00,4.510319,3.911898,3.288253,1.974531
pval,7.001856e-07,0.000010,0.000121,0.001169,0.049539
min,-2.584268e+01,-25.146664,-27.085578,-29.274522,-32.335606
25%,-1.113525e+00,-1.674382,-1.919636,-1.905503,-2.659431
50%,1.760518e+00,1.599702,1.895067,1.795390,1.355737
75%,4.712622e+00,4.644428,4.861434,4.762927,4.455583
max,1.302830e+01,13.132692,14.294360,13.889250,19.994535


In [64]:
port['spread'] = port[4] - port[0]
summary(port)

bins,0,1,2,3,4,spread
count,2.270000e+02,227.000000,227.000000,227.000000,227.000000,227.000000
mean,1.620887e+00,1.451746,1.350776,1.227299,0.850794,-0.770093
std,4.783370e+00,4.849491,5.202459,5.623387,6.491920,3.553456
tstat,5.105421e+00,4.510319,3.911898,3.288253,1.974531,-3.265165
pval,7.001856e-07,0.000010,0.000121,0.001169,0.049539,0.001264
min,-2.584268e+01,-25.146664,-27.085578,-29.274522,-32.335606,-15.190250
25%,-1.113525e+00,-1.674382,-1.919636,-1.905503,-2.659431,-2.502008
50%,1.760518e+00,1.599702,1.895067,1.795390,1.355737,-0.835878
75%,4.712622e+00,4.644428,4.861434,4.762927,4.455583,0.651610
max,1.302830e+01,13.132692,14.294360,13.889250,19.994535,19.998845


In [65]:
stks = df.groupby(['mdt','bins'])['permno'].nunique()
stks = stks.groupby('bins').mean()
stks

bins
0    516.079295
1    514.273128
2    514.264317
3    514.044053
4    514.211454
Name: permno, dtype: float64

In [66]:
disp_lag3 = stk.merge(ibes, on = ['mdt','cusip'], how = 'inner')

#create dispersion var
df['disp'] = df['stdev'] / abs(df['meanest'])
#shift by 3
df['prclag'] = df.groupby('permno')['prc'].shift(3)
df['disp'] = df.groupby('permno')['disp'].shift(3)
#drop stocks with price less than 5
df = df.query("disp == disp and prclag>= 5").reset_index(drop=True)
#cut into bins
df['bins'] = df.groupby('mdt')['disp'].transform(pd.qcut,5,labels = False)

In [67]:
port = df.groupby(['mdt','bins'])['ret'].mean()*100
port = port.unstack(level='bins')
port['spread'] = port[4] - port[0]
summary(port)

bins,0,1,2,3,4,spread
count,224.000000,224.000000,224.000000,224.000000,224.000000,224.000000
mean,1.516394,1.457117,1.318824,1.189612,1.013650,-0.502744
std,4.721734,4.807913,5.114928,5.482905,6.430015,3.543457
tstat,4.806561,4.535882,3.858968,3.247271,2.359391,-2.123459
pval,0.000003,0.000009,0.000149,0.001345,0.019168,0.034817
min,-25.573172,-24.789132,-27.067985,-29.000757,-32.724670,-12.800181
25%,-1.069470,-1.607506,-1.948081,-1.975457,-2.207026,-2.077426
50%,1.647387,1.975342,1.795436,1.542001,1.539702,-0.597608
75%,4.513865,4.515331,4.511227,4.590964,4.770858,1.247318
max,12.376284,12.507646,14.621796,14.366671,20.402470,20.372775


The results are almost identical and all the returns are statistically different from 0. Meaning that the dispersion effect is consistent through time.